
# Table of Contents

1.  [Intro to SQLAlchemy](#orgcd988ed)
    1.  [Database Primer](#org726aad9)
        1.  [Relational Database](#org8516126)
        2.  [SQL the language](#orge344737)
        3.  [Table Design](#org0cb102f)
        4.  [Resources](#org8724301)
    2.  [SQLA: The database toolkit](#org6bb0ded)
        1.  [SqlAlchemy Core](#org50ce13c)
        2.  [SqlAlchemy ORM](#org6e0c581)


<a id="orgcd988ed"></a>

# Intro to SQLAlchemy


<a id="org726aad9"></a>

## Database Primer


<a id="org8516126"></a>

### Relational Database

[RDBMS](https://en.wikipedia.org/wiki/Relational_database)

-   data storage
    -   structured
    -   efficient
    -   correct
-   primitives
    -   table
        -   columns
            -   name
            -   data type
            -   constrain value
                -   nullable
                -   unique
                -   foreign key
    -   rows
    -   index
-   server based
    -   postgres
    -   mysql/mariahdb
    -   oracle
    -   microsoft sql server
    -   db2
-   file based
    -   sqlite


<a id="orge344737"></a>

### SQL the language

[SQL](https://en.wikipedia.org/wiki/SQL)(structured query language)

-   DQL: data query language
    -   ex: select column from table where column = value
-   DDL: data definition language
    -   ex: create table&#x2026;, alter table&#x2026;
-   DML: data manipulation language
    -   ex: insert&#x2026;, update&#x2026;
-   DCL: data control language
    -   ex: grant&#x2026;, revoke&#x2026;

[more information](https://www.geeksforgeeks.org/sql-ddl-dql-dml-dcl-tcl-commands/)


<a id="org0cb102f"></a>

### Table Design

-   [normalization](https://en.wikipedia.org/wiki/Database_normalization)
    -   Fourth Normal Form is ideal
    -   Third Normal Form is practical
-   relationships
    -   uses foreign keys
    -   establish a kind of relationship 
        -   one to many
        -   many to one
        -   many to many


<a id="org8724301"></a>

### Resources

-   [sql tutorial](https://www.sqltutorial.org/)
-   [use the index luke](https://use-the-index-luke.com/)


<a id="org6bb0ded"></a>

## SQLA: The database toolkit

-   not a port of hibernate to python
-   powerful
-   flexible
-   battle tested
-   source code is a prime example of the power of python
-   maintained by ONE PERSON
-   [the best docs](https://docs.sqlalchemy.org/en/13/)
-   [prevent sql injection](https://xkcd.com/327/)


<a id="org50ce13c"></a>

### SqlAlchemy Core

-   connect to database
-   execute sql statements on connection and get results
-   Table objects
    -   define them
    -   create them
    -   read existing table from a database
    -   build sql statements programatically


<a id="org6e0c581"></a>

### SqlAlchemy ORM

-   ORM = Object Relational Mapper
    -   class represents a table
    -   class instance represents a row in the table
    -   related queries represented as properties
-   built on core
-   allows for queries to return class instances instead of dict like



# Music Database
We all know what music is right? How would we model a database to hold the information?


For the simplicity's sake, let's ignore collaborations and compilations and model things like this...

ARTIST has one to many TRACKs
ALBUM has one to many TRACKS
ARTIST has one to many ALBUMs
TRACK has one ARTIST
TRACK has one ALBUM

So we need to create 3 tables
ARTIST
ALBUM
TRACK

but first, lets connect to a database

In [ ]:
import sqlalchemy as sa

DB_NAME="music.db"
engine = sa.create_engine(f"sqlite:///{DB_NAME}")
meta = sa.MetaData()

# Define Tables with SqlAlchemy

In [ ]:
# artist table
tbl_artist = sa.Table('artist', meta,
                  sa.Column('id',
                            sa.Integer, 
                            primary_key=True, 
                            autoincrement=True),
                 sa.Column('name', 
                           sa.String, 
                           unique=True, 
                           nullable=False))
# album table
tbl_album = sa.Table('album', meta, 
                    sa.Column('id',
                             sa.Integer,
                             primary_key=True,
                             autoincrement=True),
                    sa.Column('name', 
                              sa.String, 
                             nullable=False),
                    sa.Column('year', 
                              sa.String), 
                    sa.Column('artist_id', 
                              sa.Integer,
                             sa.ForeignKey('artist.id')))
# track table
tbl_track = sa.Table('track', meta,
                    sa.Column('id', 
                              sa.Integer, 
                              primary_key=True,
                              autoincrement=True),
                    sa.Column('path', 
                             sa.String,
                             unique=True,
                             nullable=False),
                    sa.Column('artist_id', 
                              sa.Integer, 
                              sa.ForeignKey('artist.id'),
                              nullable=False),
                    sa.Column('album_id', 
                              sa.Integer,
                              sa.ForeignKey('album.id')),
                     sa.Column('name', 
                               sa.String, 
                               nullable=False),
                    sa.Column('track_number', 
                              sa.Integer),
                    sa.Column('genre', sa.String))
meta.drop_all(bind=engine)
meta.create_all(bind=engine)

# read in music data prepared by the tags.py file on my collection which took about 10 minutes


In [ ]:
import json
music_json = None
with open('music.json', 'r') as f:
    music_json = json.loads(f.read())

In [ ]:
music_json[0]

# need to get data from music_json into the artist, album and track tables

In [ ]:
for i, track_json in enumerate(music_json[0:999]):
    with engine.begin() as connection:
        # insert artist if not exists
        tag_artist_name = track_json['tag_data'].get('artist', [''])[0]
    
        # check for existence with query
        artist_exists = tbl_artist.select().where(
            tbl_artist.c.name==tag_artist_name
        )
        
        db_result = connection.execute(artist_exists).first()
        
        if db_result:
            artist_id = db_result['id']
        else:
            ins_artist = tbl_artist.insert().values(name=tag_artist_name)
            db_result = connection.execute(ins_artist)
            artist_id = db_result.lastrowid

        # insert album if not exists
        tag_album_name = track_json['tag_data'].get('album', [''])[0]
        tag_album_year = track_json['tag_data'].get('date', [''])[0]
        
        album_exists = tbl_album.select().where(sa.and_(
            tbl_album.c.artist_id==artist_id,
            tbl_album.c.name==tag_album_name,
            tbl_album.c.year==tag_album_year
        ))
        
        db_result = connection.execute(album_exists).first()
        
        if db_result:
            album_id = db_result['id']
        else:
            ins_album = tbl_album.insert().values(
                name=tag_album_name,
                year=tag_album_year,
                artist_id=artist_id)
            db_result = connection.execute(ins_album)
            album_id = db_result.lastrowid
        
        # insert track if not exists
        tag_track_name = track_json['tag_data']['title'][0]
        tag_track_path = track_json['file']
        tag_track_track_number = track_json['tag_data'].get('tracknumber', [''])[0]
        tag_track_genre = track_json['tag_data'].get('genre', [''])[0]
        
        track_exists = tbl_track.select().where(
            tbl_track.c.path==tag_track_path
        )
        
        db_result = connection.execute(track_exists).first()
        
        if not db_result:
            ins_track = tbl_track.insert().values(
                artist_id=artist_id,
                album_id=album_id,
                path=tag_track_path,
                name=tag_track_name,
                track_number=tag_track_track_number,
                genre=tag_track_genre            
            )
            connection.execute(ins_track)

print('done importing')

In [ ]:
engine.execute("select * from artist").fetchall()

# So what about the ORM?
As mentioned, sqlalchemy also comes with an object relational mapper or (ORM), which allows you to map classes to tables, and instances of those classes are mapped to rows, these kinds of classes are typically called models. It's debateable as to whether this is beneficial or not. Some tend to shy away from it because of the bad experience they had using hibernate(one of the first orm libraries) in java. MY OPINION is, that on those projects we tended to implement all the apps behaviour in the models. In general, as a rule classes, like functions should do a few well understood things rather than doing lots of things.

Nonetheless, ORMs can be useful and as a rule of thumb I will try to keep the behaviour models implement retricted to behaviour associated with a row or the corresponding table. 

In [ ]:
import sqlalchemy as sa
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship

ModelBase = declarative_base()

# tables are identical but named different so that previous 
# cells continue to work

class Artist(ModelBase):
    __tablename__ = 'artist_model'
    id = sa.Column(sa.Integer, 
                   primary_key=True, 
                   autoincrement=True)
    name = sa.Column(sa.String, 
                     unique=True, 
                     nullable=False)
    albums = relationship('Album', backref='artist')
    tracks = relationship('Track', backref='artist')
    
class Album(ModelBase):
    __tablename__ = 'album_model'
    id = sa.Column(sa.Integer,
                   primary_key=True,
                   autoincrement=True)
    name = sa.Column(sa.String, 
                     nullable=False)
    year = sa.Column(sa.String)
    artist_id = sa.Column(sa.Integer,
                          sa.ForeignKey('artist_model.id'))
    tracks = relationship('Track', backref='album')

class Track(ModelBase):
    __tablename__ = 'track_model'
    id = sa.Column(sa.Integer, 
                   primary_key=True,
                   autoincrement=True)
    path = sa.Column(sa.String,
                     unique=True,
                     nullable=False)
    artist_id = sa.Column(sa.Integer, 
                          sa.ForeignKey('artist_model.id'),
                          nullable=False)
    album_id = sa.Column(sa.Integer,
                         sa.ForeignKey('album_model.id'))
    name = sa.Column(sa.String, 
                     nullable=False)
    track_number = sa.Column(sa.Integer)
    genre = sa.Column(sa.String)
    
ModelBase.metadata.create_all(engine)

So far, looks pretty similar to the previous example where we were only declaring tables. Out of the box the models cannot do much because they do not have a database to talk to. sqla provides a Session object for this purpose.

In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine) # session is now pointing to our database as configured for the engine we created previously

# inserting rows with models

In [ ]:
# create an instance of the model and set attributes
new_artist = Artist(name='Rage Against the Machine')
print(new_artist.id, new_artist.name)

In [ ]:
session = Session()
session.add(new_artist)
session.commit()

# querying

In [ ]:
session = Session()
artist = session.query(Artist).first()

In [ ]:
print(artist.name, artist.id, artist.albums)

# updating rows

In [ ]:
session = Session()
existing_artist = session.query(Artist).filter(Artist.id == 1).one()
existing_artist.name = f"Audio Slave (formerly {existing_artist.name})"
session.commit()

session = Session()
changed_existing_artist = session.query(Artist).filter(Artist.id == existing_artist.id).one()
print(changed_existing_artist.name)
session.commit()

# How about relationships?

In [ ]:
session = Session()
artist = session.query(Artist).first()
artist.albums.append(Album(name='Audioslave', year='2002'))
session.commit()

In [ ]:
Session().query(Album).count()